In [34]:
%%capture
%load_ext autoreload
%autoreload 2

# Windows Environment Simulation
from dotenv import load_dotenv
load_dotenv()

### (ExPred) Load the pretrained model

In [35]:
%%capture
from src.expred import (seeding, ExpredInput,
                        BertTokenizerWithSpans, ExpredConfig, Expred)
from src.expred.models import (prepare_for_cl, train_evidence_classifier,
                               train_mtl_token_identifier)
from transformers import BertTokenizer

# or, simply
expred_config = ExpredConfig(
    pretrained_dataset_name='fever',
    base_dataset_name='fever',
    device='cpu',
    load_from_pretrained=True)

# seeding
seeding(1234)

# Initialize tokenizer
tokenizer = BertTokenizerWithSpans.from_pretrained('bert-base-uncased')

# create the model
expred = Expred.from_pretrained(expred_config)
expred.eval()


## Evaluation of subsequences
The black-box model of ExPred that we test for short-cuts, can be applied to automate fact-checking.
The dataset curated for that purpose is called FeVer: a dataset for Fact Verification.

In [36]:
# Read subset of FeVer corresponding to all occurences of a subsequence
import pandas as pd

evaluations = pd.read_json("has_only_ever_been.json")

In [4]:
# Create a function to simplify execution of ExPred
def evaluate_query_with_expred(current_query, evidence):
    # transform the input to the way the expred accepts
    expred_input = ExpredInput(
        queries=[current_query.split()],
        docs=[evidence.split()],
        labels=['SUPPORTS', 'REFUTES'],
        config=expred_config,
        ann_ids=['spontan_1'],
        span_tokenizer=tokenizer)
    # don't forget to preprocess
    expred_input.preprocess()

    # the output is in the form of a dict:
    expred_output = expred(expred_input)

    # retrieve the evaluation label
    current_output = expred_input.get_decoded_cls_preds(expred_output)
    return [current_query, current_output[0]]

### Subsequence: 'has only ever been'

First we will confirm that the subsequence holds true for all occurrences of the subsequence.
From the observations it can be seen that all cases of the subset containing the regex are refuted.

In [37]:
# Batch size
n = 34

results = []

for i in range (0, n):
    if(evaluations['query'][i]):
        current_query = evaluations['query'][i]
        current_doc = evaluations['evidences'][i]

        results.append(evaluate_query_with_expred(current_query, current_doc))

df = pd.DataFrame(results, columns = ['query', 'label'])
df


,query,label
0,Winnie Harlow has only ever been a librarian.,REFUTES
1,Vikram has only ever been in Spanish-language ...,REFUTES
2,Desperate Housewives has only ever been a horr...,REFUTES
3,Mel Gibson has only ever been an actor.,REFUTES
4,Mel Gibson has only ever been an actor.,REFUTES
5,Mel Gibson has only ever been an actor.,REFUTES
6,Mel Gibson has only ever been an actor.,REFUTES
7,Mel Gibson has only ever been an actor.,REFUTES
8,Ron Cobb has only ever been an illustrator.,REFUTES
9,Ron Cobb has only ever been an illustrator.,REFUTES


Percentage of confirmed cases:

In [38]:
(((df.label.values == 'REFUTES').sum())/n)*100

100.0

### Unseen Claims

Now, we will feed the Expred model three new queries containing the subsequence, with supported evidence.
The next three claims are supported, but as the output of ExPred reads the model regards them to be false nonetheless.

**Claim 1:**
`` "The city of Rome has only ever been the capital of the Kingdom of Italy." ``

**Evidence supporting claim 1:**
`` "Rome has been the national capital of Italy since 1871, when the Kingdom of Italy was established." ``

**Claim 2:**
`` "The president of the United States has only ever been a male." ``

**Evidence supporting claim 2:**
`` "United States has never had a female president." ``

**Claim 3:**
`` "Tom Hanks has only ever won Academy Awards for Best Actor." ``

**Evidence supporting claim 3:**
`` "Tom Hanks has won two Academy Awards, both in the category of Best Actor." ``

In [12]:
r1 = []

test_query_one = "The city of Rome has only ever been the capital of the Kingdom of Italy."
evidence_query_one = "Rome has been the national capital of Italy since 1871, when the Kingdom of Italy was established."
r1.append(evaluate_query_with_expred(test_query_one, evidence_query_one))

test_query_two = "The president of the United States has only ever been a male."
evidence_query_two = "United States has never had a female president."
r1.append(evaluate_query_with_expred(test_query_two, evidence_query_two))

test_query_three = "Tom Hanks has only ever been winning Academy Awards for Best Actor."
evidence_query_three = "Tom Hanks has won two Academy Awards, both in the category of Best Actor."
r1.append(evaluate_query_with_expred(test_query_three, evidence_query_three))

pd.DataFrame(r1, columns = ['query', 'label'])

,query,label
0,The city of Rome has only ever been the capita...,REFUTES
1,The president of the United States has only ev...,REFUTES
2,Tom Hanks has only ever been winning Academy A...,REFUTES


# Adverserial Attacks

Now we change the phrasing of the three unseen queries, take out the subsequence, and replace it by wording that keeps the meaning the same.

As a result, the evidence now points to Supported labels for the same questions that were asked before.

**Claim 1** 
`` 'The city of Rome was the constant capital of the Kingdom of Italy.' ``

**Claim 2** 
`` 'The president of the United States has consistently been a male.' ``

**Claim 3**  
`` 'Tom Hanks consistently won Academy Awards for Best Actor.' ``

In [13]:
r2 = []

test_query_one = "The city of Rome was the constant capital of the Kingdom of Italy."
evidence_query_one = "Rome has been the national capital of Italy since 1871, when the Kingdom of Italy was established."
r2.append(evaluate_query_with_expred(test_query_one, evidence_query_one))

test_query_two = "The president of the United States has consistently been a male."
evidence_query_two = "United States has never had a female president."
r2.append(evaluate_query_with_expred(test_query_two, evidence_query_two))

test_query_three = "Tom Hanks consistently won Academy Awards for Best Actor."
evidence_query_three = "Tom Hanks has won two Academy Awards, both in the category of Best Actor."
r2.append(evaluate_query_with_expred(test_query_three, evidence_query_three))

pd.DataFrame(r1, columns = ['query', 'label'])

,query,label
0,The city of Rome has only ever been the capita...,REFUTES
1,The president of the United States has only ev...,REFUTES
2,Tom Hanks has only ever been winning Academy A...,REFUTES


In [46]:
substring = "has only ever been"
replacewith = "has consistently been"
n = len(evaluations)

results = []

for i in range (0, n):
    current_query = evaluations['query'][i].replace(substring, replacewith)
    current_doc = evaluations['evidences'][i]

    results.append(evaluate_query_with_expred(current_query, current_doc))

df = pd.DataFrame(results, columns = ['query', 'label'])
df

,query,label
0,Winnie Harlow has consistently been a librarian.,SUPPORTS
1,Vikram has consistently been in Spanish-langua...,SUPPORTS
2,Desperate Housewives has consistently been a h...,REFUTES
3,Mel Gibson has consistently been an actor.,SUPPORTS
4,Mel Gibson has consistently been an actor.,SUPPORTS
...,...,...
243,Macbeth has consistently been a play.,SUPPORTS
244,Roman Atwood has consistently been Russian.,REFUTES
245,Roman Atwood has consistently been a beauty vl...,SUPPORTS
246,Roman Atwood has consistently been a beauty vl...,SUPPORTS


## Conclusion

This experiment means that 'has only ever been' is a <span style="color:green">valid short-cut for the ExPred model</span>, as it has very high probability that ExPred refutes a query containing this subsequence. The probability is acquired as follows:

In [48]:
(((df.label.values == 'SUPPORTS').sum())/n)*100

61.693548387096776